In [1]:
import scapy
from scapy.all import sniff, get_if_list, conf
from collections import Counter
import pandas as pd
import socket
import os

In [2]:
pd.options.display.width = 100

In [3]:
conf.promisc = 1

In [4]:
print(conf.promisc)

1


In [5]:
def get_packets(interface = "ethernet_0", filename = "sniff_cap.csv", count = 100, print_status = False, timeout=None):
    """
    This function is designed to capture a number of ip packets to analyize with pandas.
    """
    #Checks if file exists
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        return df
    #Sniffs on all interfaces for 'count' number of ip packets
    else:    
        sniffed_packets = sniff(iface=interface, count=count, filter="ip", timeout=timeout)
        list_of_packets = []
        i = 0

        for packet in sniffed_packets:
            if packet.haslayer('IP'):
                try:
                    source_name = socket.gethostbyaddr(packet['IP'].src)
                except socket.herror:
                    source_name = None
                try:
                    destination_name = socket.gethostbyaddr(packet['IP'].dst)
                except socket.herror:
                    destination_name = None
                i += 1
                #Keeps track of how many captures and prints to screen
                if print_status == True:
                    if i % (count/10) == 0:
                        print(f"Percent complete: {(i/count) * 100} %")

                #Collects the information into a dictionary for later use in Pandas
                temp_dict = {"src_IP" : str(packet['IP'].src),
                             "src_port" : str(packet['TCP'].sport),
                             "src_name": source_name,
                             "dst_IP" : str(packet['IP'].dst),
                             "dst_port" : str(packet['TCP'].dport),
                             "dst_name": destination_name,
                             "payload": str(packet['IP'].payload)}
                list_of_packets.append(temp_dict)
                
        df = pd.DataFrame(list_of_packets)
        df.to_csv(filename, index=False)

        return df

In [7]:
list_of_ifaces0 = get_if_list()

In [8]:
print(list_of_ifaces0)

['{217DAD13-5201-4EEE-B205-4F31065CF761}', '{7A07D573-0071-441B-A7D2-D3CAFCA16C04}', '{1C6779D3-9CD1-444E-B66B-7EDD47FAC11A}', '{0219B8EB-FE94-4EA5-82BF-C99143F73652}', '{ADEDECC7-19BC-4058-AF14-B3A979C5CD8B}', '{1DDD1D6E-5B88-4289-99CC-49CD5461A0AA}', '{88C13772-5A61-444B-AB31-1E707AF98A1C}', '{C22433C0-0D4C-4A08-8461-BFDA55258DE6}', '{90C98F97-8012-47A5-8B3F-95BAEFCD49DB}', '{3599E5CE-E006-4731-89A4-B15BAC330E10}', '\\Device\\NPF_Loopback', '{2E13AFA1-E1D8-4E8F-A8FE-B66EA56EC4E9}', '{AFDEECBA-DFBA-CAFF-5044-013412BCEACD}', '{786174BC-E58A-4658-9AFE-432011E9A766}']


In [9]:
usable_ifaces = []

In [10]:
for iface in list_of_ifaces0:
    try:
        iface = iface.replace("{", "").replace("}", "")
        print(iface)
        sniff(iface=iface, count=5, timeout=10)
        usable_ifaces.append(iface)
    except OSError:
        pass

217DAD13-5201-4EEE-B205-4F31065CF761
7A07D573-0071-441B-A7D2-D3CAFCA16C04
1C6779D3-9CD1-444E-B66B-7EDD47FAC11A
0219B8EB-FE94-4EA5-82BF-C99143F73652
ADEDECC7-19BC-4058-AF14-B3A979C5CD8B
1DDD1D6E-5B88-4289-99CC-49CD5461A0AA
88C13772-5A61-444B-AB31-1E707AF98A1C
C22433C0-0D4C-4A08-8461-BFDA55258DE6
90C98F97-8012-47A5-8B3F-95BAEFCD49DB
3599E5CE-E006-4731-89A4-B15BAC330E10
\Device\NPF_Loopback
2E13AFA1-E1D8-4E8F-A8FE-B66EA56EC4E9
AFDEECBA-DFBA-CAFF-5044-013412BCEACD
786174BC-E58A-4658-9AFE-432011E9A766


In [11]:
#print(usable_ifaces)

In [12]:
i = 0
for item in usable_ifaces:
    print(item)
    sniffer = get_packets(interface=str(item), filename = f"sniff_{i}.csv", timeout=60)
    #print(sniffer['src_port'].value_counts().head(3))
    #print(sniffer['dst_port'].value_counts().head(3))
    print("-----------------------------------------------------------")
    i+= 1
print(f"Usable interfaces: {i}")

\Device\NPF_Loopback
-----------------------------------------------------------
Usable interfaces: 1


In [13]:
#df.shape